# Progetto Applicazioni Data Intensive 2023
## Riconoscimento di un'anomalia

#### Autori: Alessandro Sciarrillo e Niccolò Mussoni

In [1]:
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn

# import necessari
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import Pipeline

In [2]:
dataset_full = pd.read_csv("../../Downloads/MetroPT3(AirCompressor).csv", sep=",", index_col="timestamp", parse_dates=["timestamp"])
#dataset_full = pd.read_csv("MetroPT3(AirCompressor).csv", sep=",", index_col="timestamp", parse_dates=["timestamp"])

In [3]:
# dall'1/02/2020 al 1/09/2020
dataset = dataset_full.drop(dataset_full.columns[[0]], axis=1)
dataset.head(2)
dataset.tail(2)

,TP2,TP3,H1,DV_pressure,Reservoirs,Oil_temperature,Motor_current,COMP,DV_eletric,Towers,MPG,LPS,Pressure_switch,Oil_level,Caudal_impulses
timestamp,,,,,,,,,,,,,,,
2020-09-01 03:59:40,-0.012,8.876,8.864,-0.022,8.878,59.550,0.0450,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0
2020-09-01 03:59:50,-0.014,8.860,8.848,-0.022,8.864,59.475,0.0425,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0


In [38]:
#aggiungiamo una colonna che indica un anomalia o meno in base ai dati giudicati da un esperto
def set_anomaly(anomaly_dates):
    dataset["anomaly"] = 0
    for s, e in anomaly_dates:
        #mask = (dataset.index > datetime.datetime(2020, 4, 12, 11, 50, 0)) & (dataset.index <= datetime.datetime(2020, 4, 12, 23, 30, 0))
        start = [int(st) for st in s.split(',')]
        end = [int(en) for en in e.split(',')]
        mask = (dataset.index >= datetime.datetime(*start)) & (dataset.index <= datetime.datetime(*end))
        dataset["anomaly"][mask] = 1 
        
anomaly_dates = [
    ('2020,4,12,11,50,0', '2020,4,12,23,30,0'),
    ('2020,4,18,0,0,0', '2020,4,18,23,59,0'),
    ('2020,4,19,0,0,0', '2020,4,19,1,30,0'),
    ('2020,4,29,3,20,0', '2020,4,29,4,0,0'),
    ('2020,4,29,22,0,0', '2020,4,29,22,20,0'),
    ('2020,5,13,14,0,0', '2020,5,13,23,59,0'),
    ('2020,5,18,5,0,0', '2020,5,18,5,30,0'),
    ('2020,5,19,10,10,0', '2020,5,19,11,0,0'),
    ('2020,5,19,22,10,0', '2020,5,19,23,59,0'),
    ('2020,5,20,0,0,0', '2020,5,20,20,0,0'),
    ('2020,5,23,9,50,0', '2020,5,23,10,10,0'),
    ('2020,5,29,23,30,0', '2020,5,29,23,59,0'),
    ('2020,5,30,0,0,0', '2020,5,30,6,0,0'),
    ('2020,6,1,15,0,0', '2020,6,1,15,40,0'),
    ('2020,6,3,10,0,0', '2020,6,3,11,0,0'),
    ('2020,6,5,10,0,0', '2020,6,5,23,59,0'),
    ('2020,6,6,0,0,0', '2020,6,6,23,59,0'),
    ('2020,6,7,0,0,0', '2020,6,7,14,30,0'),
    ('2020,7,8,17,30,0', '2020,7,8,19,0,0'),
    ('2020,7,15,14,30,0', '2020,7,15,19,0,0'),
    ('2020,7,17,4,30,0', '2020,7,17,5,30,0'),
]

set_anomaly(anomaly_dates)

/tmp/ipykernel_11822/979877573.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset["anomaly"][mask] = 1


,TP2,TP3,H1,DV_pressure,Reservoirs,Oil_temperature,Motor_current,COMP,DV_eletric,Towers,MPG,LPS,Pressure_switch,Oil_level,Caudal_impulses,anomaly
timestamp,,,,,,,,,,,,,,,,
2020-06-05 23:59:10,8.364,8.156,-0.008,2.218,8.158,75.700,5.6775,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0
2020-06-05 23:59:20,8.386,8.166,-0.008,2.232,8.168,75.650,5.6575,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0
2020-06-05 23:59:30,8.388,8.178,-0.008,2.232,8.178,75.500,5.5825,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0
2020-06-05 23:59:40,8.394,8.186,-0.008,2.238,8.188,75.500,5.5125,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0
2020-06-05 23:59:50,5.798,8.190,-0.008,1.354,8.190,75.475,5.0325,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0
2020-06-06 00:00:00,7.512,8.174,-0.008,1.944,8.176,75.600,5.4650,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1
2020-06-06 00:00:10,8.246,8.160,-0.008,2.178,8.162,75.700,5.6275,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1
2020-06-06 00:00:20,8.348,8.166,-0.006,2.218,8.166,75.700,5.5150,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1
2020-06-06 00:00:30,8.346,8.172,-0.006,2.220,8.172,75.500,5.5000,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1


In [137]:
dataset["COMP"].astype(np.uint8)

timestamp
2020-02-01 00:00:00    1
2020-02-01 00:00:10    1
2020-02-01 00:00:19    1
2020-02-01 00:00:29    1
2020-02-01 00:00:39    1
                      ..
2020-09-01 03:59:10    1
2020-09-01 03:59:20    1
2020-09-01 03:59:30    1
2020-09-01 03:59:40    1
2020-09-01 03:59:50    1
Name: COMP, Length: 1516948, dtype: uint8

In [163]:
all_features = {
    "TP2": tp2,
    "TP3": dataset["TP3"],
    "H1": dataset["H1"],
    "DV_pressure": dataset["DV_pressure"],
    "Reservoirs": dataset["Reservoirs"],
    "Oil_temperature": dataset["Oil_temperature"],
    "Motor_current": dataset["Motor_current"],
    
    "COMP": dataset["COMP"].astype(np.uint8),
    "DV_eletric": dataset["DV_eletric"].astype(np.uint8),
    "Towers": dataset["Towers"].astype(np.uint8),
    "MPG": dataset["MPG"].astype(np.uint8),
    "LPS": dataset["LPS"].astype(np.uint8),
    "Pressure_switch": dataset["Pressure_switch"].astype(np.uint8),
    "Oil_level": dataset["Oil_level"].astype(np.uint8),
    "Caudal_impulses": dataset["Caudal_impulses"].astype(np.uint8),
}

analog_features = {
    "TP2": tp2,
    "TP3": dataset["TP3"],
    "H1": dataset["H1"],
    "DV_pressure": dataset["DV_pressure"],
    "Reservoirs": dataset["Reservoirs"],
    "Oil_temperature": dataset["Oil_temperature"],
    "Motor_current": dataset["Motor_current"],
}
X = pd.DataFrame(analog_features)

y = dataset["anomaly"].astype(np.uint8)

X.head(5)

,TP2,TP3,H1,DV_pressure,Reservoirs,Oil_temperature,Motor_current
timestamp,,,,,,,
2020-02-01 00:00:00,-0.012,9.358,9.340,-0.024,9.358,53.600,0.0400
2020-02-01 00:00:10,-0.014,9.348,9.332,-0.022,9.348,53.675,0.0400
2020-02-01 00:00:19,-0.012,9.338,9.322,-0.022,9.338,53.600,0.0425
2020-02-01 00:00:29,-0.012,9.328,9.312,-0.022,9.328,53.425,0.0400
2020-02-01 00:00:39,-0.012,9.318,9.302,-0.022,9.318,53.475,0.0400


In [200]:
from sklearn.model_selection import train_test_split
period = X.index.month == 5
#period &= X.index.month <=6
X_train, X_val, y_train, y_val = train_test_split(
    X.loc[period],
    y.loc[period],
    test_size=1/3, random_state=42
)

X_train.index.isin(X_val.index).sum()

y_val.value_counts(normalize=True)

0    0.932825
1    0.067175
Name: anomaly, dtype: float64

In [187]:
model = LinearRegression()
model.fit(X_train, y_train)
model.score(X_val, y_val)

0.8493996005889551

In [178]:
model.predict(X_val)

array([ 0.0370609 , -0.01686998,  0.02771862, ..., -0.02337393,
       -0.01863149,  0.03825194])

In [179]:
#model.score(mired_X_val, mired_Y_val)
#model.predict(mired_X_val).mean()
from sklearn.metrics import classification_report
prediction = model.predict(X_val)
prediction = [0 if i<=0.5 else 1 for i in prediction]
print(classification_report(y_val, prediction))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00     39741
           1       0.91      0.98      0.95      2819

    accuracy                           0.99     42560
   macro avg       0.96      0.99      0.97     42560
weighted avg       0.99      0.99      0.99     42560



In [186]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver="saga", random_state=42)
model.fit(X_train, y_train)
model.score(X_val, y_val)

/home/sciro/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


0.9944548872180451

In [21]:
from sklearn.svm import OneClassSVM
one_class_svm = OneClassSVM(nu=0.01, kernel = 'rbf', gamma = 'auto').fit(X_train)

In [26]:
from sklearn.metrics import classification_report
# Predict the anomalies
prediction = one_class_svm.predict(X_val)
# Change the anomalies' values to make it consistent with the true values
prediction = [1 if i==-1 else 0 for i in prediction]
# Check the model performance
print(classification_report(y_val, prediction))

              precision    recall  f1-score   support

           0       0.93      0.98      0.96    198605
           1       0.58      0.24      0.34     17909

    accuracy                           0.92    216514
   macro avg       0.76      0.61      0.65    216514
weighted avg       0.91      0.92      0.91    216514



In [171]:
model = Pipeline([
    ("poly",    PolynomialFeatures(degree=3, include_bias=False) ),
    ("regr", Ridge(alpha=1.0) )
])

model.fit(X_train, y_train)
model.score(X_val, y_val)

0.9445412597745251

In [175]:
from sklearn.metrics import classification_report
prediction = model.predict(X_val)
prediction = [0 if i<=0.5 else 1 for i in prediction]
print(classification_report(y_val, prediction))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     39741
           1       0.96      0.98      0.97      2819

    accuracy                           1.00     42560
   macro avg       0.98      0.99      0.99     42560
weighted avg       1.00      1.00      1.00     42560



In [183]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver="saga", random_state=42)
model.fit(X_train, y_train)
print("Non Regolarizzato:",model.score(X_val,y_val))

# regolarizzato
model = LogisticRegression(solver="saga", random_state=42, penalty="l1", C=0.01)
model.fit(X_train, y_train);
print("Regolarizzato:",model.score(X_val,y_val))

#elasticnet
model = LogisticRegression(
    solver="saga", random_state=42,
    penalty="elasticnet", C=0.1, l1_ratio=0.2)
model.fit(X_train, y_train);
print("Elasticnet:",model.score(X_val,y_val))

/home/sciro/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Non Regolarizzato: 0.9944548872180451


/home/sciro/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Regolarizzato: 0.9947603383458646
Elasticnet: 0.9944783834586466


/home/sciro/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [191]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier(max_depth=1)
model.fit(X_train, y_train);
model.score(X_val,y_val)

0.9921757518796992

In [192]:
prediction = model.predict(X_val).astype(np.uint8)
print(classification_report(y_val, prediction))
#print(y_val.dtype, prediction.dtype)

              precision    recall  f1-score   support

           0       1.00      0.99      1.00     39741
           1       0.91      0.98      0.94      2819

    accuracy                           0.99     42560
   macro avg       0.95      0.99      0.97     42560
weighted avg       0.99      0.99      0.99     42560



In [168]:
import xgboost
from xgboost import XGBClassifier 
model = XGBClassifier(objective="reg:squarederror", learning_rate=0.0157, reg_lambda= 0.008, reg_alpha=0.0001, n_estimators=625) 
model.fit(X_train, y_train)
model.score(X_val, y_val)

0.9979793233082707

In [169]:
prediction = model.predict(X_val)
print(classification_report(y_val, prediction))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     39741
           1       0.99      0.98      0.98      2819

    accuracy                           1.00     42560
   macro avg       0.99      0.99      0.99     42560
weighted avg       1.00      1.00      1.00     42560



## Contesto e Obbiettivo del modello
(Linee guida: Descrivere in modo chiaro il contesto e l’obiettivo del modello di predizione, la fonte e la struttura del
dataset utilizzato e le variabili che contiene. Eseguire eventualmente una prima scrematura dei dati,
eliminando ad es. variabili palesemente non informative (es. identificatori) o con molti dati nulli.)

## Analisi Esplorativa
(Linee guida: Eseguire un’analisi esplorativa del dataset, riportando statistiche generali (medie, quartili, valori distinti,
indici di correlazione, …) e distribuzioni delle variabili in tabelle e grafici (a torta, istogrammi, a dispersione,
…). Commentare adeguatamente i risultati dell’analisi ed utilizzarli eventualmente per eliminare dati non
utilizzabili nell’analisi.)

In [16]:
dataset.describe()

,Unnamed: 0,TP2,TP3,H1,DV_pressure,Reservoirs,Oil_temperature,Motor_current,COMP,DV_eletric,Towers,MPG,LPS,Pressure_switch,Oil_level,Caudal_impulses
count,1.516948e+06,1.516948e+06,1.516948e+06,1.516948e+06,1.516948e+06,1.516948e+06,1.516948e+06,1.516948e+06,1.516948e+06,1.516948e+06,1.516948e+06,1.516948e+06,1.516948e+06,1.516948e+06,1.516948e+06,1.516948e+06
mean,7.584735e+06,1.367826e+00,8.984611e+00,7.568155e+00,5.595619e-02,8.985233e+00,6.264418e+01,2.050171e+00,8.369568e-01,1.606106e-01,9.198483e-01,8.326640e-01,3.420025e-03,9.914368e-01,9.041556e-01,9.371066e-01
std,4.379053e+06,3.250930e+00,6.390951e-01,3.333200e+00,3.824015e-01,6.383070e-01,6.516261e+00,2.302053e+00,3.694052e-01,3.671716e-01,2.715280e-01,3.732757e-01,5.838091e-02,9.214078e-02,2.943779e-01,2.427712e-01
min,0.000000e+00,-3.200000e-02,7.300000e-01,-3.600000e-02,-3.200000e-02,7.120000e-01,1.540000e+01,2.000000e-02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,3.792368e+06,-1.400000e-02,8.492000e+00,8.254000e+00,-2.200000e-02,8.494000e+00,5.777500e+01,4.000000e-02,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
50%,7.584735e+06,-1.200000e-02,8.960000e+00,8.784000e+00,-2.000000e-02,8.960000e+00,6.270000e+01,4.500000e-02,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
75%,1.137710e+07,-1.000000e-02,9.492000e+00,9.374000e+00,-1.800000e-02,9.492000e+00,6.725000e+01,3.807500e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
max,1.516947e+07,1.067600e+01,1.030200e+01,1.028800e+01,9.844000e+00,1.030000e+01,8.905000e+01,9.295000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


## Preparazione Dati
(Linee guida: Preparare i dati per l’addestramento e la validazione dei modelli di predizione: isolare la variabile “target”
da predire e le variabili predittive, suddividere i dati in training e test set, eseguire eventuali operazioni di
preprocessing come ad es. one-hot encoding di variabili categoriche e oversampling o undersampling in
caso di classi sbilanciate.)

## Addestramento e Validazione
(Linee guida: Addestrare e validare due o più modelli di predizione, calcolandone le misure di performance viste nel
corso (es. MSE, errore relativo e coefficiente R² per modelli di regressione) e analizzando il modello
addestrato (es. coefficienti in una regressione lineare o nodi dei primi livelli di un albero decisionale) per
individuare le variabili più o meno rilevanti nella predizione.)

## Ricerca Iperparametri
(Linee guida: Scegliere uno o più modelli di base (es. regressione ridge) ed eseguire una ricerca degli iperparametri
esaustiva (grid search) o a campione (randomized search) che massimizzi le performance del modello.)